# Day 22 - Card game implementation

This is a pretty straightforward coding exercise; I used [`collection.deque()` objects](https://docs.python.org/3/library/collections.html#collections.deque) to hold the card numbers as they are more efficient and easier to work with when continuing to add and remove values from either end.

In [1]:
from collections import deque
from dataclasses import dataclass
from itertools import islice
from typing import Deque


@dataclass
class Hand:
    player: int
    cards: Deque[int]

    @property
    def score(self) -> int:
        return sum(i * c for i, c in enumerate(reversed(self.cards), 1))

    @classmethod
    def from_data(cls, data: str) -> "Hand":
        lines = data.splitlines()
        player = int(lines[0][7:-1])
        return cls(player, deque([int(c) for c in lines[1:]]))


@dataclass
class Game:
    player1: Hand
    player2: Hand

    @classmethod
    def from_data(cls, data: str) -> "Game":
        return cls(*map(Hand.from_data, data.split("\n\n")))

    def play(self) -> Hand:
        p1, p2 = self.player1, self.player2
        p1c, p2c = p1.cards, p2.cards
        while p1c and p2c:
            c1, c2 = p1c.popleft(), p2c.popleft()
            if c1 > c2:
                p1c += c1, c2
            else:
                p2c += c2, c1
        return p1 if p1c else p2


test_data = """\
Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10
"""
test_game = Game.from_data(test_data)
assert test_game.play().score == 306

In [2]:
import aocd
data = aocd.get_data(day=22, year=2020)

In [3]:
game = Game.from_data(data)
print("Part 1:", game.play().score)

Part 1: 31629


## Part 2

For part 2, I created a subclass that simply creates more instances of itself to run the recursive subgames:()

In [4]:
class RecursiveCombatGame(Game):
    player1: Hand
    player2: Hand

    def play(self) -> Hand:
        p1, p2 = self.player1, self.player2
        p1c, p2c = p1.cards, p2.cards
        seen = set()
        while p1c and p2c:
            if (state := (tuple(p1c), tuple(p2c))) in seen:
                # infinite loop, p1 always wins
                return p1
            seen.add(state)
            c1, c2 = p1c.popleft(), p2c.popleft()
            if c1 <= len(p1c) and c2 <= len(p2c):
                p1wins = self._recurse(c1, c2)
            else:
                p1wins = c1 > c2

            if p1wins:
                p1c += c1, c2
            else:
                p2c += c2, c1

        return p1 if p1c else p2

    def _recurse(self, c1, c2):
        p1 = Hand(1, deque(islice(self.player1.cards, c1)))
        p2 = Hand(2, deque(islice(self.player2.cards, c2)))
        return RecursiveCombatGame(p1, p2).play().player == 1


test_rcombat = RecursiveCombatGame.from_data(test_data)
assert test_rcombat.play().score == 291

In [5]:
rcombat = RecursiveCombatGame.from_data(data)
print("Part 2:", rcombat.play().score)

Part 2: 35196
